## Linear Program with Monetized Grid Emissions Dataframe

This linear program builds on the resourcedf script by incorporating monetized health impacts from hourly grid emissions.

In [136]:
%reload_ext autoreload
%autoreload 2

import numpy as np # numerical library
import matplotlib.pyplot as plt # plotting library
import datetime as dt
import pandas as pd

from ortools.linear_solver import pywraplp

In [137]:
import utils

In [138]:
emissionsdf_solver = pywraplp.Solver('HarborOptimization',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

#Introduce objective object so we can refer to it in the for loop.
objective = emissionsdf_solver.Objective()

In [139]:
# Load Harbor historical hourly generation and emissions for 2014-2018. Group by datetime to sum generation and emissions from all units. Filter for specific year -- make sure selected year isn't leap year.
harborgen = utils.get_harbor_data('data/HarborHourly_2014-18.csv')
harborgen = harborgen.groupby(['datetime'])['mwh'].sum()

# Create string object that is the chosen year
year = '2018'

# Filter Harbor data by the chosen year, using the string object above.
harborgen = harborgen.filter(like=year, axis=0)

In [140]:
# Load generation profiles for nondispatchable resources (KWh generated each hour by 1 KW of capacity).
profiles = pd.read_csv('data/gen_profiles.csv')

# Match profiles index to harborgen index. **Currently this only works for one year -- for multiple years, need to repeat gen profiles.
profiles.insert(0, 'datetime', harborgen.index)
profiles = profiles.set_index('datetime')

In [141]:
profiles.head()

,solar,ee_1,ee_2,ee_3,ee_4,ee_5,ee_6,ee_7,ee_8,ee_9,ee_10
datetime,,,,,,,,,,,
2018-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
harborgen.head()

datetime
2018-01-01 00:00:00    0.0
2018-01-01 01:00:00    0.0
2018-01-01 02:00:00    0.0
2018-01-01 03:00:00    0.0
2018-01-01 04:00:00    0.0
Name: mwh, dtype: float64

In [143]:
resources = pd.read_csv('data/resource_costs.csv')
resources = resources.set_index('resource')

In [144]:
resources

,new,fossil,dispatchable,capex,fixed,variable
resource,,,,,,
gas_harbor,n,y,y,2.0,NaN,0.21
gas_repower,n,y,y,5.0,NaN,0.10
solar,y,n,n,6.0,NaN,0.00
demand_response,y,n,y,4.0,NaN,0.15
storage_utility,y,n,y,2.0,NaN,0.05
storage_res,y,n,y,1.0,NaN,0.06
storage_ci,y,n,y,2.0,NaN,0.06
storage_diesel,y,n,y,3.0,NaN,0.06
ee_1,y,n,n,5.0,NaN,0.00


In [145]:
grid_emissions = pd.read_csv('data/grid_emissions.csv')
grid_emissions.insert(0, 'datetime', harborgen.index)
grid_emissions = grid_emissions.set_index('datetime')
grid_emissions

,CO2,NOX,SO2,PM2.5,PM10,CH4,TOTAL/MWH
datetime,,,,,,,
2018-01-01 00:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-01-01 01:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-01-01 02:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-01-01 03:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-01-01 04:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
...,...,...,...,...,...,...,...
2018-12-31 19:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-12-31 20:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-12-31 21:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5


In [146]:
# Declare nameplate capacity variables for each resource in resource cost dataframe.
capacity_vars = {}
for resource in resources.index:
    capacity = emissionsdf_solver.NumVar(0, emissionsdf_solver.infinity(), str(resource))
    capacity_vars[resource] = capacity
    
capacity_vars

{'gas_harbor': gas_harbor,
 'gas_repower': gas_repower,
 'solar': solar,
 'demand_response': demand_response,
 'storage_utility': storage_utility,
 'storage_res': storage_res,
 'storage_ci': storage_ci,
 'storage_diesel': storage_diesel,
 'ee_1': ee_1,
 'ee_2': ee_2,
 'ee_3': ee_3,
 'ee_4': ee_4,
 'ee_5': ee_5,
 'ee_6': ee_6,
 'ee_7': ee_7,
 'ee_8': ee_8,
 'ee_9': ee_9,
 'ee_10': ee_10}

In [147]:
#Create filtered dataframes for dispatchable and nondispatchable resources.
disp = resources.loc[resources['dispatchable'] == 'y']
nondisp = resources.loc[resources['dispatchable'] == 'n']
disp

,new,fossil,dispatchable,capex,fixed,variable
resource,,,,,,
gas_harbor,n,y,y,2.0,NaN,0.21
gas_repower,n,y,y,5.0,NaN,0.10
demand_response,y,n,y,4.0,NaN,0.15
storage_utility,y,n,y,2.0,NaN,0.05
storage_res,y,n,y,1.0,NaN,0.06
storage_ci,y,n,y,2.0,NaN,0.06
storage_diesel,y,n,y,3.0,NaN,0.06


In [148]:
#Create a dictionary to hold a list for each dispatchable resource that keeps track of its hourly generation variables.
disp_gen = {}
for resource in disp.index:
    disp_gen[resource] = []

In [149]:
disp_gen

{'gas_harbor': [],
 'gas_repower': [],
 'demand_response': [],
 'storage_utility': [],
 'storage_res': [],
 'storage_ci': [],
 'storage_diesel': []}

In [158]:
#Loop through every hour, creating 1) hourly generation variables for each dispatchable resource, 2) hourly constraints, and 3) adding variable cost coefficients to each hourly generation variable.
for ind in harborgen.index:
    
    #Summed generation from all resources must be equal or greater to demand in all hours.
    fulfill_demand = emissionsdf_solver.Constraint(harborgen.loc[ind], emissionsdf_solver.infinity())
    
    #Create generation variable for each dispatchable resource for every hour. Append hourly gen variable to the list for that resource, located in the disp_gen dictionary.
    #Create constraint that generation must be less than or equal to capacity for each dispatchable resource for all hours.
    for resource in disp.index:
        
        max_gen = emissionsdf_solver.Constraint(0, emissionsdf_solver.infinity())
        gen = emissionsdf_solver.NumVar(0, emissionsdf_solver.infinity(), '_gen'+ str(ind))
        disp_gen[resource].append(gen)
        
        # **** Need to fix this so grid emissions are only avoided for clean resources. Should this only be for additional energy generated beyond Harbor?
        variable_cost = disp.loc[resource,'variable'] - grid_emissions.loc[ind,'TOTAL/MWH']
        objective.SetCoefficient(gen, variable_cost)
        
        #Set coefficients for the hourly gen variables for the fulfill_demand constraint.
        fulfill_demand.SetCoefficient(gen, 1)
        
        #Set coefficients for dispatchable capacity variables and hourly gen variables for the max_gen = capacity constraint. =
        max_gen.SetCoefficient(capacity_vars[resource], 1)
        max_gen.SetCoefficient(gen, -1)
        
    
    #For each nondispatchable resource, set the coefficient of the capacity variable to its generation profile scaling factor for the "fulfill demand" constraint. **Make sure units are aligned here (kw vs. mw capacities)
    for resource in nondisp.index: 
        coefficient = profiles.loc[ind, resource]
        fulfill_demand.SetCoefficient(capacity_vars[resource], coefficient)
        

In [159]:
# Add capex costs as coefficients of the capacity variables in the objective function. 
# ***** Need to add in fixed costs -- assume a certain plant lifetime and amortize over that period, using discount rate. ******

# Loop through dispatchable resources, adding capex cost to capacity variables for the objective function. 
for resource in disp.index:
    capex = disp.loc[resource, 'capex']
    objective.SetCoefficient(capacity_vars[resource], capex)
    
# Loop through nondispatchable resources, adding capex cost to capacity variables and incorporating cost savings from avoided hourly grid emissions. *** The code currently subtracts cost savings from the capex cost of nondispatchable resources. This means for every MW of a nondispatchable resource built, there is a fixed amount of cost savings from avoided grid emissions associated with its generation profile.
for resource in nondisp.index:
    capex = nondisp.loc[resource, 'capex']
    profile = profiles[str(resource)]
    emissions_savings = grid_emissions['TOTAL/MWH'].dot(profile)
    objective.SetCoefficient(capacity_vars[resource], capex - emissions_savings)

In [160]:
objective.SetMinimization()
emissionsdf_solver.Solve()
print("total cost =", objective.Value())

for resource in capacity_vars:
    print(str(capacity_vars[resource]) + ' capacity =' + str(capacity_vars[resource].solution_value()))

# Sum generation for each dispatchable resource and print.
for resource in disp.index:
    summed_gen = 0
    for i in range(8760):
        summed_gen = summed_gen + disp_gen[resource][i].solution_value()
    print(str(resource) + ': annual generation =' + str(summed_gen))
    
# Sum annual generation for nondispatchable resources and print.
summed_gen = profiles.sum()
for resource in nondisp.index:
    summed_gen.filter(resource)
    capacity = capacity_vars[resource].solution_value()
    gen = summed_gen[resource] * capacity
    print(str(resource) + ': annual generation =' + str(gen))


total cost = 0.0
gas_harbor capacity =0.0
gas_repower capacity =0.0
solar capacity =0.0
demand_response capacity =0.0
storage_utility capacity =0.0
storage_res capacity =0.0
storage_ci capacity =0.0
storage_diesel capacity =0.0
ee_1 capacity =0.0
ee_2 capacity =0.0
ee_3 capacity =0.0
ee_4 capacity =0.0
ee_5 capacity =0.0
ee_6 capacity =0.0
ee_7 capacity =0.0
ee_8 capacity =0.0
ee_9 capacity =0.0
ee_10 capacity =0.0
gas_harbor: annual generation =0.0
gas_repower: annual generation =0.0
demand_response: annual generation =0.0
storage_utility: annual generation =0.0
storage_res: annual generation =0.0
storage_ci: annual generation =0.0
storage_diesel: annual generation =0.0
solar: annual generation =0.0
ee_1: annual generation =0.0
ee_2: annual generation =0.0
ee_3: annual generation =0.0
ee_4: annual generation =0.0
ee_5: annual generation =0.0
ee_6: annual generation =0.0
ee_7: annual generation =0.0
ee_8: annual generation =0.0
ee_9: annual generation =0.0
ee_10: annual generation =0.0